In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = ""

import sys

sys.path.append("../../")

import numpy as np
import pandas as pd

In [ ]:
from choice_learn.datasets import load_swissmetro
swiss_dataset = load_swissmetro(preprocessing="tutorial")
print(swiss_dataset.summary())

In [ ]:
from choice_learn.models.learning_mnl import LearningMNL

In [ ]:
# Initialization of the model
swiss_model = LearningMNL(optimizer="Adam",
nn_features=['regular_class', 'single_luggage_piece', 'multiple_luggage_piece'], nn_layers_widths=[12])

# Intercept for train & sm
swiss_model.add_coefficients(feature_name="intercept", items_indexes=[0, 1])
# beta_he for train & sm
swiss_model.add_coefficients(feature_name="headway",
                             items_indexes=[0, 1],
                             coefficient_name="beta_he")
# beta_co for all items
swiss_model.add_coefficients(feature_name="cost",
                             items_indexes=[0, 1, 2])
# beta seats for train
swiss_model.add_coefficients(feature_name="seats", items_indexes=[1])
# beta TT only for car
swiss_model.add_coefficients(feature_name="travel_time",
                             items_indexes=[2],
                             coefficient_name="beta_tt_car")

# betas TT and HE shared by train and sm
swiss_model.add_shared_coefficient(feature_name="travel_time",
                                   items_indexes=[0, 1])
swiss_model.add_shared_coefficient(feature_name="train_survey",
                                   items_indexes=[0, 1],
                                   coefficient_name="beta_survey")


In [ ]:
# Estimation of the model
history = swiss_model.fit(swiss_dataset)

In [ ]:
swiss_dataset.shared_features_by_choice